In [1]:
import pandas as pd
from utils import umap_func,extract_projections,umap_2d_scatter_chart,create_folder
import os
import joblib
pd.set_option('display.max_columns', None)


# The notebook create an embeddings representation of the text rule and show it a interactive chart

In [2]:
df=pd.read_csv("out/1_elaborated_dataset.csv")
df

,Unnamed: 0,by_service_owner,channels,description,friendly_id,id,installs_count,name,permissions,pro_features,requires_android_app,requires_ios_app,requires_mobile_app,service_names,service_slug,services,speed,uniq_permissions,services_len,service_triggers,service_actions,triggers_category,actions_category,goal,operation
0,6913,True,"[{'name': 'Button widget', 'module_name': 'do_...",NaN,wYf8ceZQ-turn-on-off-your-lights-with-one-tap-...,wYf8ceZQ,164055.0,Turn on/off your lights with one tap on your p...,[{'id': '/triggers/do_button.do_button_new_com...,False,False,False,False,"['Button widget', 'Philips Hue']",hue,"['do_button', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/do_button.do_button_new_com...,2,Button widget,Philips Hue,Mobile devices & accessories,Lighting,['Ambient Luminance'],Explicit Control (Button)
1,11608,True,"[{'name': 'Weather Underground', 'module_name'...",Never be left in the dark. Whenever the sun st...,PVkgiLYy-automatically-turn-your-lights-on-at-...,PVkgiLYy,144011.0,Automatically turn your lights on at sunset,"[{'id': '/triggers/weather.sun_sets', 'name': ...",False,False,False,False,"['Weather Underground', 'Philips Hue']",hue,"['weather', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/triggers/weather.sun_sets', 'name': ...",2,Weather Underground,Philips Hue,Weather,Lighting,['Ambient Luminance'],Time Trigger
2,27384,True,"[{'name': 'Location', 'module_name': 'location...","Center the map on your home's location, and yo...",HTak4X5f-turn-your-lights-on-automatically-as-...,HTak4X5f,69594.0,Turn your lights on automatically as you arriv...,[{'id': '/triggers/location.enter_region_locat...,False,False,False,True,"['Location', 'Philips Hue']",hue,"['location', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/location.enter_region_locat...,2,Location,Philips Hue,Location,Lighting,['Ambient Luminance'],Spatial Trigger
3,2745,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Choose a phrase to control your Harmony Home w...,sHYgpAeS-tell-alexa-to-start-a-harmony-activity,sHYgpAeS,66448.0,Tell Alexa to start a Harmony activity,[{'id': '/triggers/amazon_alexa.custom_utteran...,False,False,False,False,"['Amazon Alexa', 'Harmony']",harmony,"['amazon_alexa', 'harmony']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/amazon_alexa.custom_utteran...,2,Amazon Alexa,Harmony,Voice assistants,Smart hubs & systems,['Control Hub'],Explicit Control (Voice)
4,27491,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Add some visual flair to your countdown. Tip: ...,funBgceh-blink-your-hue-lights-when-your-amazo...,funBgceh,47628.0,Blink your Hue lights when your Amazon Alexa t...,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",False,False,False,False,"['Amazon Alexa', 'Philips Hue']",hue,"['amazon_alexa', 'hue']",Realtime Applets usually run within 10 seconds,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",2,Amazon Alexa,Philips Hue,Voice assistants,Lighting,['Alert User'],Other Trigger
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,13693,False,"[{'name': 'WeMo Light Switch', 'module_name': ...",Long press Wemo Switch = All Hue lights off,qPhRymsC-long-press-wemo-switch-all-hue-lights...,qPhRymsC,10.0,Long press Wemo Switch = All Hue lights off,[{'id': '/triggers/wemo_light_switch.attribute...,False,False,False,False,"['WeMo Light Switch', 'Philips Hue']",wemo_light_switch,"['wemo_light_switch', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/actions/hue.turn_off_all_hue', 'name...",2,WeMo Light Switch,Philips Hue,Lighting,Lighting,['Ambient Luminance'],Other Trigger
2556,9959,False,"[{'name': 'RSS Feed', 'module_name': 'feed', '...","When Adafruit releases a new product, blink!",Ktup6MV2-when-adafruit-releases-a-new-product-...,Ktup6MV2,10.0,"When Adafruit releases a new product, blink!",[{'id': '/triggers/feed.new_feed_item_matches'...,F

## Embed "name" feature through language model
start with "bert-base-uncased"

In [3]:
from sentence_transformers import SentenceTransformer

output_dir="sentence_embedding_finetuning/models/mpnet-base-all-nli-triplet-4ep" #load the finetuned model from local folder
model_embedding_name="all-mpnet-base-v2-finetuned"


In [4]:

fine_tuned_model = SentenceTransformer(output_dir)

In [5]:
model = fine_tuned_model

In [6]:
print(model)
print("Max tokens Sequence Length:", model.max_seq_length)# (longer inputs will be truncated!)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
Max tokens Sequence Length: 384


## Use the "name" column attribute as the only textual representation of the rule

In [7]:
device='cpu'

embeddings = model.encode(df["name"], device=device,output_value="sentence_embedding")# mean of the words embeddings as the representative for the sentence
embeddings.shape,embeddings# some checks

((2560, 768),
 array([[-0.038109  , -0.00789538, -0.0192273 , ..., -0.00541885,
          0.01937505,  0.01477483],
        [-0.06282733,  0.08642986, -0.01314095, ...,  0.00692127,
          0.00085832,  0.00849081],
        [-0.0460535 ,  0.03229946, -0.02238792, ...,  0.0261548 ,
         -0.00467431,  0.00573931],
        ...,
        [ 0.06073958,  0.00836326, -0.02188724, ..., -0.02586745,
          0.00613012, -0.04101495],
        [ 0.02583823,  0.07540289,  0.00200409, ..., -0.01437893,
         -0.03769761,  0.00616436],
        [-0.01689353,  0.02439754,  0.00898682, ..., -0.00699768,
         -0.05302438, -0.02829444]], dtype=float32))

In [8]:
df["embeddings"]=list(embeddings)

In [9]:
#save the calculated embeddings
emdeddings_path="out/embeddings_"+model_embedding_name
joblib.dump(embeddings, emdeddings_path)

['out/embeddings_all-mpnet-base-v2-finetuned']

In [10]:
# just a check
emdeddings_path="out/embeddings_"+model_embedding_name
embeddings = joblib.load(emdeddings_path)
embeddings,len(embeddings),embeddings[0].shape

(array([[-0.038109  , -0.00789538, -0.0192273 , ..., -0.00541885,
          0.01937505,  0.01477483],
        [-0.06282733,  0.08642986, -0.01314095, ...,  0.00692127,
          0.00085832,  0.00849081],
        [-0.0460535 ,  0.03229946, -0.02238792, ...,  0.0261548 ,
         -0.00467431,  0.00573931],
        ...,
        [ 0.06073958,  0.00836326, -0.02188724, ..., -0.02586745,
          0.00613012, -0.04101495],
        [ 0.02583823,  0.07540289,  0.00200409, ..., -0.01437893,
         -0.03769761,  0.00616436],
        [-0.01689353,  0.02439754,  0.00898682, ..., -0.00699768,
         -0.05302438, -0.02829444]], dtype=float32),
 2560,
 (768,))

## Create a visual representation 

In [11]:
#create a 2d umap projection
#https://umap-learn.readthedocs.io/en/latest/
projections_2d=umap_func(embeddings, n_components=2)
projections_2d

c:\Users\nicleonard\AppData\Local\anaconda3\envs\hiis\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


generating projections with UMAP took: 14.70 sec


array([[-3.8536897e+00, -3.2260966e-01],
       [-4.0838165e+00,  3.2937026e-03],
       [-3.4517279e+00,  1.8875964e-01],
       ...,
       [ 2.1929982e+00,  8.6951475e+00],
       [-3.3241293e+00, -7.9464418e-01],
       [-2.1702168e+00,  4.9723125e+00]], dtype=float32)

In [12]:
first_projection, second_projection=extract_projections(projections_2d)
#first_projection, second_projection

In [13]:
df["umap_2d_first"]=first_projection
df["umap_2d_second"]=second_projection

In [14]:
df

,Unnamed: 0,by_service_owner,channels,description,friendly_id,id,installs_count,name,permissions,pro_features,requires_android_app,requires_ios_app,requires_mobile_app,service_names,service_slug,services,speed,uniq_permissions,services_len,service_triggers,service_actions,triggers_category,actions_category,goal,operation,embeddings,umap_2d_first,umap_2d_second
0,6913,True,"[{'name': 'Button widget', 'module_name': 'do_...",NaN,wYf8ceZQ-turn-on-off-your-lights-with-one-tap-...,wYf8ceZQ,164055.0,Turn on/off your lights with one tap on your p...,[{'id': '/triggers/do_button.do_button_new_com...,False,False,False,False,"['Button widget', 'Philips Hue']",hue,"['do_button', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/do_button.do_button_new_com...,2,Button widget,Philips Hue,Mobile devices & accessories,Lighting,['Ambient Luminance'],Explicit Control (Button),"[-0.038109004, -0.007895378, -0.019227298, -0....",-3.853690,-0.322610
1,11608,True,"[{'name': 'Weather Underground', 'module_name'...",Never be left in the dark. Whenever the sun st...,PVkgiLYy-automatically-turn-your-lights-on-at-...,PVkgiLYy,144011.0,Automatically turn your lights on at sunset,"[{'id': '/triggers/weather.sun_sets', 'name': ...",False,False,False,False,"['Weather Underground', 'Philips Hue']",hue,"['weather', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/triggers/weather.sun_sets', 'name': ...",2,Weather Underground,Philips Hue,Weather,Lighting,['Ambient Luminance'],Time Trigger,"[-0.06282733, 0.08642986, -0.013140954, -0.043...",-4.083817,0.003294
2,27384,True,"[{'name': 'Location', 'module_name': 'location...","Center the map on your home's location, and yo...",HTak4X5f-turn-your-lights-on-automatically-as-...,HTak4X5f,69594.0,Turn your lights on automatically as you arriv...,[{'id': '/triggers/location.enter_region_locat...,False,False,False,True,"['Location', 'Philips Hue']",hue,"['location', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/location.enter_region_locat...,2,Location,Philips Hue,Location,Lighting,['Ambient Luminance'],Spatial Trigger,"[-0.046053495, 0.03229946, -0.022387922, -0.01...",-3.451728,0.188760
3,2745,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Choose a phrase to control your Harmony Home w...,sHYgpAeS-tell-alexa-to-start-a-harmony-activity,sHYgpAeS,66448.0,Tell Alexa to start a Harmony activity,[{'id': '/triggers/amazon_alexa.custom_utteran...,False,False,False,False,"['Amazon Alexa', 'Harmony']",harmony,"['amazon_alexa', 'harmony']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/amazon_alexa.custom_utteran...,2,Amazon Alexa,Harmony,Voice assistants,Smart hubs & systems,['Control Hub'],Explicit Control (Voice),"[-0.03308521, 0.029681448, -0.024129465, -0.03...",4.312790,6.452626
4,27491,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Add some visual flair to your countdown. Tip: ...,funBgceh-blink-your-hue-lights-when-your-amazo...,funBgceh,47628.0,Blink your Hue lights when your Amazon Alexa t...,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",False,False,False,False,"['Amazon Alexa', 'Philips Hue']",hue,"['amazon_alexa', 'hue']",Realtime Applets usually run within 10 seconds,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",2,Amazon Alexa,Philips Hue,Voice assistants,Lighting,['Alert User'],Other Trigger,"[-0.016805189, 0.013020454, 0.0035018132, -0.0...",0.693110,-0.931202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,13693,False,"[{'name': 'WeMo Light Switch', 'module_name': ...",Long press Wemo Switch = All Hue lights off,qPhRymsC-long-press-wemo-switch-all-hue-lights...,qPhRymsC,10.0,Long press Wemo Switch = All Hue lights off,[{'id': '/triggers/wemo_light_switch.attribute...,False,False,False,False,"['WeMo Light Switch', 'Philips Hue']",wemo_light_switch,"['wemo_light_switch', 'hue']",Polling Applets usually run within 1 hour,"[{'id':

In [15]:
#create and save a interactive scatter chart (saved into the out_folder)
umap_2d_scatter_chart(df,model_embedding_name)

### example from bert_base_uncased
<img src="figures/bert_uncased_umap_2d.png" width=1000>

From the representation (a 2d proxy of the embeddings space) it is clear how points close in the space (semantically similar from the point of view of the textual representation) have different labels associated with them (colors). The ideal would be for points close to each other to have the same labels. 
Se want to reach this result at the end of our experimentation 

## Save the elaborated dataframe

In [15]:
#create_folder(os.getcwd(),next_path="out")

In [16]:
df.to_csv("out/2_elaborated_dataset_with_embeddings_"+model_embedding_name+"_.csv", index=False)

In [17]:
# just a check
pd.read_csv("out/2_elaborated_dataset_with_embeddings_"+model_embedding_name+"_.csv")

,Unnamed: 0,by_service_owner,channels,description,friendly_id,id,installs_count,name,permissions,pro_features,requires_android_app,requires_ios_app,requires_mobile_app,service_names,service_slug,services,speed,uniq_permissions,services_len,service_triggers,service_actions,triggers_category,actions_category,goal,operation,embeddings,umap_2d_first,umap_2d_second
0,6913,True,"[{'name': 'Button widget', 'module_name': 'do_...",NaN,wYf8ceZQ-turn-on-off-your-lights-with-one-tap-...,wYf8ceZQ,164055.0,Turn on/off your lights with one tap on your p...,[{'id': '/triggers/do_button.do_button_new_com...,False,False,False,False,"['Button widget', 'Philips Hue']",hue,"['do_button', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/do_button.do_button_new_com...,2,Button widget,Philips Hue,Mobile devices & accessories,Lighting,['Ambient Luminance'],Explicit Control (Button),[-3.81090045e-02 -7.89537840e-03 -1.92272980e-...,-3.853690,-0.322610
1,11608,True,"[{'name': 'Weather Underground', 'module_name'...",Never be left in the dark. Whenever the sun st...,PVkgiLYy-automatically-turn-your-lights-on-at-...,PVkgiLYy,144011.0,Automatically turn your lights on at sunset,"[{'id': '/triggers/weather.sun_sets', 'name': ...",False,False,False,False,"['Weather Underground', 'Philips Hue']",hue,"['weather', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/triggers/weather.sun_sets', 'name': ...",2,Weather Underground,Philips Hue,Weather,Lighting,['Ambient Luminance'],Time Trigger,[-6.28273264e-02 8.64298567e-02 -1.31409541e-...,-4.083817,0.003294
2,27384,True,"[{'name': 'Location', 'module_name': 'location...","Center the map on your home's location, and yo...",HTak4X5f-turn-your-lights-on-automatically-as-...,HTak4X5f,69594.0,Turn your lights on automatically as you arriv...,[{'id': '/triggers/location.enter_region_locat...,False,False,False,True,"['Location', 'Philips Hue']",hue,"['location', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/location.enter_region_locat...,2,Location,Philips Hue,Location,Lighting,['Ambient Luminance'],Spatial Trigger,[-4.60534953e-02 3.22994590e-02 -2.23879218e-...,-3.451728,0.188760
3,2745,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Choose a phrase to control your Harmony Home w...,sHYgpAeS-tell-alexa-to-start-a-harmony-activity,sHYgpAeS,66448.0,Tell Alexa to start a Harmony activity,[{'id': '/triggers/amazon_alexa.custom_utteran...,False,False,False,False,"['Amazon Alexa', 'Harmony']",harmony,"['amazon_alexa', 'harmony']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/amazon_alexa.custom_utteran...,2,Amazon Alexa,Harmony,Voice assistants,Smart hubs & systems,['Control Hub'],Explicit Control (Voice),[-3.30852084e-02 2.96814479e-02 -2.41294652e-...,4.312790,6.452626
4,27491,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Add some visual flair to your countdown. Tip: ...,funBgceh-blink-your-hue-lights-when-your-amazo...,funBgceh,47628.0,Blink your Hue lights when your Amazon Alexa t...,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",False,False,False,False,"['Amazon Alexa', 'Philips Hue']",hue,"['amazon_alexa', 'hue']",Realtime Applets usually run within 10 seconds,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",2,Amazon Alexa,Philips Hue,Voice assistants,Lighting,['Alert User'],Other Trigger,[-1.68051887e-02 1.30204540e-02 3.50181316e-...,0.693110,-0.931202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,13693,False,"[{'name': 'WeMo Light Switch', 'module_name': ...",Long press Wemo Switch = All Hue lights off,qPhRymsC-long-press-wemo-switch-all-hue-lights...,qPhRymsC,10.0,Long press Wemo Switch = All Hue lights off,[{'id': '/triggers/wemo_light_switch.attribute...,False,False,False,False,"['WeMo Light Switch', 'Philips Hue']",wemo_light_switch,"['wemo_light_switch', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/actions/hue.